# Earth Engine data

In [1]:
from irrigation_detection import get_s1_grd_mean

In [3]:
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWh2kSs4j9h1G30nDcRgA5HxaZAUMs_ZKmeFJQEi85cRON9Y_LTk_-I

Successfully saved authorization token.


In [4]:
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_test_field.geojson'
get_s1_grd_mean(path,'2016-02-01','2021-10-31','whr_test_field', 'yes', 30)

0 (930) images between 2016-02-01 and 2021-10-31 within 0.05884062361097424 km² 

Polling for task (id: B7GDZYZS65U27G37HNYJGMPX).
finished


In [6]:
def s1_construct_gdf(p_geojson):
    """
    Arguments:
    """
    #import modules
    import geopandas as gpd
    import numpy as np
    import pandas as pd
    from datetime import datetime
    
    gdf = gpd.read_file(p_geojson)
    gdf['patch'] = [x[-1] for x in gdf.id]
    gdf['sentinel'] = [x[2] for x in gdf.id]
    gdf['date'] = [datetime.strptime(x.split('_')[4][:15], '%Y%m%dT%H%M%S') for x in gdf.id]
    gdf['orbit'] = ['ascending' if x.hour > 10 else 'descending' for x in gdf.date]
    gdf['identifier'] = [x[-6:-2] for x in gdf.id]
    gdf.to_file(p_geojson)
    return gdf.iloc[:,1:]

In [ ]:
gdf = s1_construct_gdf(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_test_field_ndvi30.geojson')
gdf = s1_construct_gdf(r'C:/Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_test_fieldG_ndvi30.geojson')

In [ ]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V20/100m/2018').select('landcover')

# Mask areas where soil moisture measurements valid (farmland cat.:11-16)
clc_mask = clc.gte(211).And(clc.lte(231)) #binary map for updateMask

In [ ]:
#create poi for irrigated field at 2020-04-26
from shapely.geometry import Point
poi = Point(8.462525010108948, 49.84003060414562)

In [ ]:
# isolate field within point
field01 = gdf[gdf.contains(poi)]
field01

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [ ]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
import hvplot
import hvplot.pandas

In [ ]:
field01.hvplot(kind='scatter', x='date.day', y='VV', by='sentinel', xticks=30)

In [ ]:
# Create imagecollection by mapping reduceToImage function over FeatureCollection
s1_grd_rr_IC = ee.ImageCollection(s1_grd_rr.map(lambda x: ee.FeatureCollection(x).filter(ee.Filter.notNull(['VV'])).reduceToImage(properties=['VV'], reducer=ee.Reducer.first()).unmask(0).reproject('epsg:4326',None,10).clip(geojsonFc.geometry())))

In [ ]:
url = s1_grd_rr_IC.first().select('first').getThumbURL({'min': -20, 'max': 1})
disp.Image(url=url, width=800)

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))

In [ ]:
s1_grd_rr_IC.first().getInfo()

In [ ]:
#location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
#rgb = ee.Image.rgb(ffa_db.select('VV'),
#                   ffa_db.select('VH'),
#                   ffa_db.select('VV').divide(ffa_db.select('VH')))
test = s1_grd_rr_IC.first()
# Create the map object.
m = folium.Map(location=['49.983461596534646','8.471317291259766'], zoom_start=15)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(test, {'min': [-20], 'max': [1]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.471317291259766,
              49.983461596534646
            ],
            [
              8.481616973876953,
              49.94867940605311
            ],
            [
              8.498611450195312,
              49.946249234682774
            ],
            [
              8.529338836669922,
              49.951882626433914
            ],
            [
              8.54710578918457,
              49.966239272192034
            ],
            [
              8.471317291259766,
              49.983461596534646
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords)


In [ ]:
hist = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_sub).get('VV').getInfo()
mean = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
a = np.array(hist)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents
plt.grid()
plt.plot(x, y, '.')
plt.show()


In [ ]:
alpha = 5
beta = mean/alpha
plt.grid()
plt.plot(x, y, '.', label='data')
plt.plot(x, gamma.pdf(x, alpha, 0, beta)/1000, '-r', label='gamma')
plt.legend()
plt.show()


In [ ]:
def X(n):
    return np.sum(np.cos(4*np.pi*(np.random.rand(n)-0.5)))/np.sqrt(n/2)

n= 10000
Xs = [X(n) for i in range(10000)]
y, x = np.histogram(Xs, 100, range=[-5,5])
plt.plot(x[:-1], y/1000, 'b.', label='simulated data')
plt.plot(x, norm.pdf(x), '-r', label='normal distribution')
plt.grid()
plt.legend()
plt.show()


In [ ]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi)
                .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 15))
                .sort('system:time_start'))


In [ ]:
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


In [ ]:
im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi_sub)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi_sub)

In [ ]:
ratio = im1.divide(im2)
url = ratio.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

In [ ]:
hist = ratio.reduceRegion(ee.Reducer.fixedHistogram(0, 5, 500), aoi_sub).get('VV').getInfo()
mean = ratio.reduceRegion(ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ratio.reduceRegion(ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi)
ratio = im1.divide(im2)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=12)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_child(folium.LayerControl())

display(mp)


In [ ]:
# Decision threshold alpha/2:
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location, tiles='Stamen Toner',
    zoom_start=13)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())

display(mp)


In [4]:
def get_s1_grd_mean(path, start, end, outname, with_ndvi, dateoffset):
    """
    Save a gejson to drive 
    Arguments: path to gejson featurecollection, start date, end date, outname, with_ndvi 'yes' or 'no', dateoffset (int) while finding correspnding ndvi values to s1 images
    """
    # Import modules.
    import ee

    try:
        # Initialize the library.
        ee.Initialize()
    except:
        # Trigger the authentication flow.
        ee.Authenticate()
        # Initialize the library.
        ee.Initialize()
    import geojson
    import geopandas as gpd
    import pandas as pd
    from glob import glob
    import os
    from datetime import datetime, timedelta
    import geemap.eefolium as geemap
    from tqdm import tqdm
    import geemap
    import time
    
    # Functions.
    # Calculate coverage in km²
    def get_area(image):
        # Count the non zero/null pixels in the image within the aoi
        actPixels = ee.Number(image.select('VV').reduceRegion(reducer= ee.Reducer.count(),scale= 10,geometry= fc_aoi.union().geometry(), maxPixels= 999999999).values().get(0))
        # calculate the perc of cover
        pcPix = actPixels.multiply(100).divide(1000000)
        return image.set('area', pcPix)
    
    #NDVI
    def add_ndvi(image):
        """
        Arguments: 
        """
        def maskS2clouds(image):
            qa = image.select('QA60')
            #Bits 10 and 11 are clouds and cirrus, respectively.
            cloudBitMask = 1 << 10
            cirrusBitMask = 1 << 11
            #Both flags should be set to zero, indicating clear conditions.
            mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
            return image.updateMask(mask).divide(10000)

        def NDVI(image):
            ndvi = image.normalizedDifference(['nir','red']).rename('NDVI') #(first − second) / (first + second)
            return image.addBands(ndvi)
        
        # Sentinel 2 image collection with corresponding named bands
        bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
        bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']
        s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2)
        s2_1c = s2_1c.filterDate(ee.Date(image.date().advance(-dateoffset,'days')), ee.Date(image.date().advance(+dateoffset,'days'))).filterBounds(image.geometry()).map(maskS2clouds).map(NDVI)
        ndvi = ee.Image(s2_1c.qualityMosaic('NDVI').select('NDVI'))

        return image.addBands(ndvi)
    
    def mask_by_ndvi(image):
        mask = image.select('NDVI').lte(0.6)
        return image.updateMask(mask)
    
    def mask_by_landcover(image):
        image = image.select('Map')
        mask = image.eq(40).Or(image.eq(30))
        return image.updateMask(mask)
    
    # Paths to initial polygon(s) and outdir for ts data.
    p_i = path
    p_o = os.path.dirname(path) + '/ts_data/'
    
    # create folder in local space when not already there.
    if not os.path.exists(p_o):
        os.makedirs(p_o)
        
    # Load aoi features from file.
    with open(p_i) as f:
        data = geojson.load(f)

    # Create GEE FeatureCollection from geojson file.
    fc_aoi = ee.FeatureCollection(data)
    area = fc_aoi.geometry().area().getInfo()
    
    fcg_aoi = ee.FeatureCollection(fc_aoi.geometry().buffer(5000))
    areag = fcg_aoi.geometry().area().getInfo()

    # Sentinel 1 GRD image collection their dates and coverage over aoi
    ic_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(fc_aoi).filterDate(ee.Date(start), ee.Date(end)).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    icg_s1 = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(fcg_aoi).filterDate(ee.Date(start), ee.Date(end)).filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))

    s1_dates = [datetime(1970, 1, 1) + timedelta(milliseconds=x) for x in ic_s1.aggregate_array("system:time_start").getInfo()]
    s1_dates = [x.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for x in s1_dates]    
    s1_coverd = ic_s1.map(get_area).aggregate_array('area').getInfo()
    
    s1g_dates = [datetime(1970, 1, 1) + timedelta(milliseconds=x) for x in ic_s1.aggregate_array("system:time_start").getInfo()]
    s1g_dates = [x.strftime("%Y-%m-%dT%H:%M:%S.%fZ") for x in s1g_dates]    
    s1g_coverd = icg_s1.map(get_area).aggregate_array('area').getInfo()
    
    # Drop low coverage by metadata filter
    s1_valid = [x for x,y in zip(s1_dates,s1_coverd) if y > area*0.25]
    s1_valid_dates = ee.List(s1_valid).map(lambda x: ee.Date(x).millis())
    #ic_s1 = ic_s1.filter(ee.Filter.inList("system:time_start", s1_valid_dates))

    # Drop low coverage by metadata filter
    s1g_valid = [x for x,y in zip(s1g_dates,s1g_coverd) if y > areag*0.25]
    s1g_valid_dates = ee.List(s1g_valid).map(lambda x: ee.Date(x).millis())
    #icg_s1 = icg_s1.filter(ee.Filter.inList("system:time_start", s1g_valid_dates))
    
    print(ic_s1.size().getInfo(),'(%s)' %(len(s1_dates)), 'images between %s and %s' %(start,end), 'within %s km²' %(area/1000000),'\n') #s1_plot.aggregate_array("system:time_start").getInfo()
    print(icg_s1.size().getInfo(),'(%s)' %(len(s1g_dates)), 'images between %s and %s' %(start,end), 'within %s km²' %(areag/1000000),'\n') #s1_plot.aggregate_array("system:time_start").getInfo()

    #Mask out area with invalid landcover class like build up areas
    lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(fc_aoi.union().geometry()).select('Map'))
    ic_s1 = ic_s1.map(lambda x: x.addBands(lc))
    ic_s1 = ic_s1.map(mask_by_landcover)
    
    #Mask out area with invalid landcover class like build up areas
    lc = ee.Image(ee.ImageCollection("ESA/WorldCover/v100").first().clip(fcg_aoi.union().geometry()).select('Map'))
    icg_s1 = icg_s1.map(lambda x: x.addBands(lc))
    icg_s1 = icg_s1.map(mask_by_landcover)
    
    if with_ndvi == 'yes':
        # Add ndvi band
        ic_s1 = ic_s1.map(add_ndvi)

        # Mask areas with ndvi > 0.6
        ic_s1 = ic_s1.map(mask_by_ndvi)
        
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    else:
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fc_s1 = ic_s1.map(lambda x: x.reduceRegions(collection=fc_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    
    if with_ndvi == 'yes':
        # Add ndvi band
        icg_s1 = icg_s1.map(add_ndvi)

        # Mask areas with ndvi > 0.6
        icg_s1 = icg_s1.map(mask_by_ndvi)
        
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fcg_s1 = icg_s1.map(lambda x: x.reduceRegions(collection=fcg_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    else:
        # Map reducer function over imagecollection to get mean for multipolygon geometries
        fcg_s1 = icg_s1.map(lambda x: x.reduceRegions(collection=fcg_aoi ,reducer='mean', crs='EPSG:4326',scale=10)).flatten()
    
    # Export the FeatureCollection to a KML file.
    task1 = ee.batch.Export.table.toDrive(collection = fc_s1,description='vectorsToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix=outname)
    task1.start()
    
    while task1.active():
      print('Polling for task (id: {}).'.format(task1.id))
      time.sleep(15)
        
    # Export the FeatureCollection to a KML file.
    task2 = ee.batch.Export.table.toDrive(collection = fcg_s1,description='vectorsGToDrive',folder='idm_gee_export', fileFormat= 'GeoJSON', fileNamePrefix=outname + 'G')
    task2.start()
    
    while task2.active():
      print('Polling for task (id: {}).'.format(task2.id))
      time.sleep(15)

    return print("finished")

In [5]:
path = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/whr_test_field.geojson'
get_s1_grd_mean(path,'2016-02-01','2021-10-31','whr_test_field', 'yes', 20)

930 (930) images between 2016-02-01 and 2021-10-31 within 0.05884062361097424 km² 

1181 (930) images between 2016-02-01 and 2021-10-31 within 82.98391916648124 km² 

Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: P64IYL62ZSCM6ZA47MAN3COG).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
Polling for task (id: ZWGAZZ75L7FRGYFY5WMGNU3K).
